# 🎯 YOLO Object Detection for Aerial Objects

## 📋 Project Overview
This notebook implements **YOLOv8-based object detection** for identifying and localizing aerial objects (birds and drones) in images. Unlike classification which only predicts categories, object detection provides both **classification** and **precise bounding box localization**.

## 🎯 Objectives
1. **Data Quality Assurance**: Clean dataset by removing corrupted or mislabeled samples
2. **YOLO Model Training**: Train YOLOv8 for aerial object detection  
3. **Performance Evaluation**: Assess detection accuracy using mAP metrics
4. **Model Deployment**: Prepare trained model for real-world applications

## 📊 Dataset Structure
- **Images**: Aerial photographs containing birds and drones
- **Labels**: YOLO format annotations with bounding boxes and class labels
- **Classes**: 2 classes (Bird=0, Drone=1)
- **Format**: Each image paired with corresponding .txt annotation file

---

## 🔍 Stage 1: Dataset Integrity Verification

### Why Data Verification is Critical:
- **Quality Assurance**: Ensure every image has corresponding labels
- **Training Stability**: Prevent errors during YOLO training due to missing files
- **Resource Efficiency**: Avoid wasting compute on corrupted data
- **Performance Impact**: Clean data directly improves model performance

### Verification Process:
1. Check dataset directory structure exists
2. Verify image-label file pairing
3. Identify and handle empty annotation files
4. Remove corrupted or incomplete samples

In [1]:
import os

print(os.path.exists("../data/object_detection_Dataset/train/images"))
print(os.path.exists("../data/object_detection_Dataset/train/labels"))
print(os.path.exists("../data/object_detection_Dataset/data.yaml"))

True
True
True


### ✅ Verify Dataset Structure

**Purpose**: Confirm all required directories and files exist before training.
**Critical Check**: Ensures data.yaml configuration file exists and points to correct paths.

In [2]:
import os

img_dir = "../data/object_detection_Dataset/train/images"
lbl_dir = "../data/object_detection_Dataset/train/labels"

removed = 0

for img in os.listdir(img_dir):
    name = os.path.splitext(img)[0]
    if not os.path.exists(os.path.join(lbl_dir, name + ".txt")):
        os.remove(os.path.join(img_dir, img))
        removed += 1

print("Removed:", removed, "images with no labels")


Removed: 0 images with no labels


### 🧹 Remove Images Without Labels

**Purpose**: Clean training data by removing images that lack corresponding label files.
**Why Important**: YOLO training fails if images don't have annotation files.
**Safety Measure**: This prevents training interruptions and ensures data consistency.

In [3]:
import os

lbl_dir = "../data/object_detection_Dataset/train/labels"

empty = []

for f in os.listdir(lbl_dir):
    path = os.path.join(lbl_dir, f)
    if os.path.getsize(path) == 0:
        empty.append(f)

len(empty), empty[:10]


(0, [])

### 📝 Identify Empty Label Files

**Purpose**: Find label files with no annotations (0 bytes or empty content).
**Problem**: Empty labels indicate images with no objects, which may cause training issues.
**Analysis**: Count and inspect empty files before deciding on removal strategy.

In [4]:
import os

train_img_dir = "../data/object_detection_Dataset/train/images"
train_lbl_dir = "../data/object_detection_Dataset/train/labels"

removed = 0

for txt in empty:
    img_base = txt.replace(".txt", "")

    # possible extensions
    for ext in [".jpg", ".png", ".jpeg"]:
        img_path = os.path.join(train_img_dir, img_base + ext)
        if os.path.exists(img_path):
            os.remove(img_path)

    # remove the label file
    os.remove(os.path.join(train_lbl_dir, txt))
    removed += 1

print("Removed:", removed, "empty-label training samples")


Removed: 0 empty-label training samples


### 🗑️ Clean Empty Label Samples

**Purpose**: Remove both images and labels for samples with empty annotations.
**Strategy**: For each empty label file, find and remove corresponding image file(s).
**Multi-format Support**: Handles different image extensions (.jpg, .png, .jpeg).
**Data Consistency**: Maintains perfect image-label pairing after cleanup.

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data="../data/object_detection_Dataset/data.yaml",
    epochs=40,
    imgsz=640,
    batch=16,
    device=0
)



Ultralytics 8.3.228  Python-3.13.5 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/object_detection_Dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, p

---

## 🚀 Stage 2: YOLO Model Training

### Why YOLOv8 for Aerial Object Detection:
- **State-of-the-Art Performance**: Latest YOLO architecture with improved accuracy
- **Real-time Inference**: Fast detection suitable for real-world applications  
- **Efficient Architecture**: YOLOv8n (nano) provides good balance of speed and accuracy
- **Transfer Learning**: Pre-trained on COCO dataset provides strong initialization

### Training Configuration:
- **Model**: YOLOv8n (nano version for efficiency)
- **Epochs**: 40 epochs for thorough training
- **Image Size**: 640x640 pixels (standard YOLO input)
- **Batch Size**: 16 (balanced for memory efficiency)
- **Device**: GPU acceleration when available

### Expected Outcomes:
- **mAP Metrics**: Mean Average Precision for detection performance
- **Training Curves**: Loss and metric progression over epochs
- **Validation Results**: Performance on held-out validation set

---

## 📊 Stage 3: Model Evaluation & Analysis

### Post-Training Analysis:
After training completion, the following analysis should be performed:

1. **Training Metrics Visualization**:
   - Loss curves (box, object, class losses)
   - mAP@0.5 and mAP@0.5:0.95 progression
   - Precision and recall curves

2. **Test Set Evaluation**:
   - Final mAP scores on test dataset
   - Per-class performance analysis
   - Inference speed benchmarking

3. **Visual Validation**:
   - Sample detection results on test images
   - Error analysis on misdetections
   - Confidence threshold optimization

4. **Model Deployment Preparation**:
   - Export trained model weights
   - Optimize for inference speed
   - Create inference pipeline

### Key Performance Indicators:
- **mAP@0.5**: Detection accuracy at 50% IoU threshold
- **mAP@0.5:0.95**: Average mAP across IoU thresholds 0.5-0.95
- **Precision**: Percentage of correct detections
- **Recall**: Percentage of actual objects detected
- **Inference Speed**: FPS for real-time applications

*Note: Training results and detailed analysis will appear here after running the training cell above.*